In [1]:
import pandas as pd
import numpy as np
import mysql.connector

Drop Review MYSQL Table if exists

In [2]:
db_connection = mysql.connector.connect(user="admin", password="yeet")
db_cursor = db_connection.cursor()
db_cursor.execute("USE cs179g;")
db_cursor.execute("DROP TABLE IF EXISTS Review;")
db_cursor.execute("DROP TABLE IF EXISTS BusinessSentiment;")


Read from parquet file

In [3]:
reivew_df = pd.read_parquet('/home/ubuntu/data/pred_avg.parquet.gzip')

Times the average prediction column by 10 to get the rating

In [4]:

reivew_df["avg(prediction)"] *= 10
reivew_df["avg(prediction)"] = reivew_df["avg(prediction)"].round(2)
reivew_df.rename(columns = {'avg(prediction)':'Rating'}, inplace = True)
display(reivew_df)


# Connect to MYSQL Database
db_connection = mysql.connector.connect(user="admin", password="yeet")
db_cursor = db_connection.cursor()
db_cursor.execute("USE cs179g;")


,business_id,Rating
0,2y_CdkxEOJEJGyJApfCYpA,2.97
1,jIBjZcqVOU4pOVT-sVQcfA,4.36
2,TuRy46Cyb7MWjV7VM8VNYg,3.25
3,Ryt1Fhgz7sixMQSJiLyP_g,3.55
4,q6661I3CGW0UB740E5Zjfw,3.91
...,...,...
150341,E9vGiqTPzu1_DQ85K0J3Lw,8.00
150342,9dPyvmwBmyAEkxd5BkRSuQ,8.00
150343,II1ZGMmA0BLGV-EmhPkrEA,3.33
150344,g48DB4oZZU3mD5D1e6l2VA,4.00


Iterate throught the dataframe and insert into the MYSQL Review database

In [5]:

# Create Review Table in MYSQL Database
db_cursor.execute("CREATE TABLE IF NOT EXISTS Review(business_id VARCHAR(255), Rating VARCHAR(5));")

for index, row in reivew_df.iterrows():
    temp2 = []
    for i in row:
        if(len(str(i)) > 255):
            temp = str(i)
            temp = temp[:255]
            temp2.append(str(temp))
        else:
            temp2.append(str(i))
    temp2 = tuple(temp2)
    command = "INSERT INTO Review (business_id, Rating) VALUES (%s, %s);"
    db_cursor.execute(command, temp2)




In [6]:
db_cursor.execute("FLUSH TABLES;")
db_cursor.execute("SELECT * FROM Review LIMIT 5;")
print(type(db_cursor.fetchall()))

<class 'list'>


Join Business and Review Table

In [7]:
db_cursor.execute("CREATE TABLE BusinessSentiment AS \
(SELECT Business.name, Business.postal_code, Business.latitide, Business.longitude, Review.Rating \
FROM Business \
INNER JOIN Review ON Review.business_id=Business.business_id);")

ProgrammingError: 1050 (42S01): Table 'BusinessSentiment' already exists